In [2]:
import numpy as np
from ipynb.fs.full.Ej7 import sol_cuadmin
import matplotlib.pyplot as plt
from scipy.optimize import linprog

In [ ]:
x=np.arange(1, 11, dtype=float)
y=np.arange(1, 11, dtype=float)
y[-1]=0
print(x.shape)

In [4]:
def sol_a(x, y):
    #Defino el sistema. A = (x,1) es una matriz, a.T = (a1, a0) es un vector. Se resuelve A.a=y
    #A es 10x2, su primera columna es el vector x y su segunda columna son todos 1
    #a es 2x1, a1 es la pendiente y a0 es la ordenada al origen
    n= x.shape[0]
    A= np.ones((n,2))
    A[:, 0]= x
    b= y.copy()
    pendiente, ord_orig= sol_cuadmin(A, b)[0]  #=np.polyfit(x,y)
    return (pendiente, ord_orig)

In [5]:
"""
Hay que minimizar (c.T)y, sujeto a:

[A, -I] [x]  [b]
[A, -I] [y]<=[-b]
donde x:n, y:m, A:mx2, b:m, c.T=[1,...1]:m pero en 
este caso lo haremos c.T=[0,0,1,...,1]:m+2
donde las primeras dos coordenadas van a 
representar a la pendiente y ordenada al origen
"""

def sol_b(x,y):
    #Creo el sistema
    #matriz
    m, n= y.shape[0], 2  #El 2 viene de la cantidad de columnas de la martriz A de la funcion anterior
    A=np.zeros((2*m, n+m), dtype=float)    
    A[:m, 0], A[:m, 1]= x, np.ones(m)
    A[m:, 0], A[m:, 1] = -x, -np.ones(m)
    A[:m,n:]=-np.eye(m)
    A[m:,n:]=-np.eye(m)
    #vector de costo y vector b´
    c=np.ones(m+2)
    c[:2]=0
    b=np.zeros(2*m)
    b[:m], b[m:]= y, -y
    #Reselvo
    sol=linprog(c, A, b).x #el .x te devuelve la solución
    pendiente, ord_orig= sol[0], sol[1]
    return pendiente, ord_orig

In [6]:
"""
Hay que minimizar c.T, sujeto a:

[A, -e] [x]  [b]
[A, -e] [t]<=[-b]
donde x:n, t:1, A:mx3, b:m, c.T=[0,0,t]:3
donde las primeras dos coordenadas van a 
representar a la pendiente y ordenada al origen
"""
def sol_c(x, y):
    #Creo el sistema
    #matriz
    m, n= x.shape[0], 2  #El 2 viene de la cantidad de columnas de la martriz A de la funcion anterior
    A=np.zeros((2*m, 2+1), dtype=float)    
    A[:m, 0], A[:m, 1]= x, np.ones(m)
    A[m:, 0], A[m:, 1] = -x, -np.ones(m)
    A[:m,-1]=-np.ones(m)
    A[m:,-1]=-np.ones(m)
    #vector de costo y vector b
    c=np.array([0, 0, 1], dtype=float)
    b=np.zeros(2*m)
    b[:m], b[m:]= y, -y
    #Reselvo
    sol=linprog(c, A, b).x #el .x te devuelve la solución
    pendiente, ord_orig= sol[0], sol[1]
    return pendiente, ord_orig


In [7]:
def grafico(): #mejor usar polyval
    a1, b1= sol_a(x,y)
    a2, b2= sol_b(x,y)
    a3, b3= sol_c(x,y)

    x1 = np.linspace(-1, 11, 200)

    fig, ax= plt.subplots()
    plt.plot(x, y,"o",color="black", label="puntos")
    plt.plot(x1, a1*x1 + b1,color="orange", label="norma2")
    plt.plot(x1, a2*x1 + b2,color="green", label="norma1")
    plt.plot(x1, a3*x1 + b3,color="blue", label="normainf")
    
    ax.set_xlabel("Eje X")
    ax.set_ylabel("Eje Y")
    ax.legend()
    plt.show()

In [ ]:
grafico()